In [3]:
# import packages and change directory
import pandas as pd

import os

os.chdir('/home/workgroups/socialclass')

os.getcwd()



'/home/workgroups/socialclass'

In [4]:
true_pairs = pd.read_stata("./data/A1_1_couples_info.dta")
men_df = pd.read_stata("./data/A1_1_couples_info_man.dta")
women_df = pd.read_stata("./data/A1_1_couples_info_woman.dta")

true_pairs = true_pairs[true_pairs['t_match'] == 2007]
men_df = men_df[men_df['t_match'] == 2007]
women_df = women_df[women_df['t_match'] == 2007]

In [5]:
import pandas as pd
import numpy as np

# men_df and women_df must already exist, with a shared key structure
# For example:
# men_df  : id, lan, yob, income, educ, ...
# women_df: id, lan, yob, income, educ, ...
# true_pairs: DataFrame with columns ['man_id', 'woman_id'] for actual marriages

# --- parameters ---
K = 10
rng = np.random.default_rng(42)

# 1️⃣  Add suffixes for clarity when merging later
# men_df = men_df.add_suffix('_m')
# women_df = women_df.add_suffix('_w')

true_pairs['match'] = 1



In [6]:

# # 2️⃣  Create the positive (true) pairs
# true_pairs = true_pairs.merge(men_df, left_on='man_id', right_on='id_m')
# true_pairs = true_pairs.merge(women_df, left_on='woman_id', right_on='id_w')

# 3️⃣  Efficient negative sampling (for men first)
def sample_negatives(df_source, df_target, source_lang_col, target_lang_col, K):
    """
    For each language group, create K random pairings across df_source and df_target with the same language.
    """
    samples = []
    for lang, men_group in df_source.groupby(source_lang_col):
        women_pool = df_target[df_target[target_lang_col] == lang]
        if women_pool.empty:
            continue
        # Randomly sample K women for each man (with replacement)
        # This creates a cartesian product (men_group × sample of women)
        idx = rng.choice(women_pool.index, size=K * len(men_group), replace=True)
        women_sample = women_pool.loc[idx].reset_index(drop=True)
        men_repeated = men_group.loc[men_group.index.repeat(K)].reset_index(drop=True)
        pairs = pd.concat([men_repeated.reset_index(drop=True),
                           women_sample.reset_index(drop=True)], axis=1)
        pairs['match'] = 0
        samples.append(pairs)
    return pd.concat(samples, ignore_index=True)

# 4️⃣  Apply for men-based negatives
neg_men = sample_negatives(men_df, women_df, 'lan1', 'lan2', K)

# # 5️⃣  Apply for women-based negatives
# neg_women = sample_negatives(women_df, men_df, 'lan_w', 'lan_m', K)

# # rename columns for consistency (we want man_* and woman_* names)
# neg_women = neg_women.rename(columns=lambda c: c.replace('_w', '_tmp') if c.endswith('_w') else c)
# neg_women = neg_women.rename(columns=lambda c: c.replace('_m', '_w') if c.endswith('_m') else c)
# neg_women = neg_women.rename(columns=lambda c: c.replace('_tmp', '_m') if c.endswith('_tmp') else c)

# # 6️⃣  Combine all
# full_pairs = pd.concat([true_pairs, neg_men, neg_women], ignore_index=True)

# print(full_pairs.shape)
# print(full_pairs['match'].value_counts())


/tmp/ipykernel_800485/568904416.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for lang, men_group in df_source.groupby(source_lang_col):


In [7]:
neg_men

,id_spouse,parent1,parent11,parent12,lob1,lob11,lob12,cob1,cob11,cob12,...,kommun21,kommun22,wealth2,wealth21,wealth22,educ2,educ21,educ22,t_match,match
0,63.0,3708472.0,11597345.0,1905940.0,1.0,21.0,4.0,Sweden,Sweden,Sweden,...,NaN,0181,1999000.0,NaN,0.0,2.0,NaN,NaN,2007,0
1,63.0,3708472.0,11597345.0,1905940.0,1.0,21.0,4.0,Sweden,Sweden,Sweden,...,0180,2480,-1207.0,-82642.0,89009.0,3.0,6.0,6.0,2007,0
2,63.0,3708472.0,11597345.0,1905940.0,1.0,21.0,4.0,Sweden,Sweden,Sweden,...,1402,1402,731791.0,1769569.0,312239.0,6.0,1.0,1.0,2007,0
3,63.0,3708472.0,11597345.0,1905940.0,1.0,21.0,4.0,Sweden,Sweden,Sweden,...,NaN,0331,-43303.0,NaN,381358.0,4.0,6.0,3.0,2007,0
4,63.0,3708472.0,11597345.0,1905940.0,1.0,21.0,4.0,Sweden,Sweden,Sweden,...,0180,0180,277.0,-6484.0,-8699.0,4.0,1.0,2.0,2007,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
544345,16217894.0,6646446.0,14552017.0,12954774.0,25.0,25.0,25.0,Sweden,Sweden,Sweden,...,0665,0665,335971.0,1560370.0,3193890.0,6.0,5.0,5.0,2007,0
544346,16217894.0,6646446.0,14552017.0,12954774.0,25.0,25.0,25.0,Sweden,Sweden,Sweden,...,2580,2580,2884216.0,130803.0,176967.0,2.0,3.0,3.0,2007,0
544347,16217894.0,6646446.0,14552017.0,12954774.0,25.0,25.0,25.0,Sweden,Sweden,Sweden,...,2581,2581,-74829.0,238871.0,161262.0,4.0,3.0,3.0,2007,0
544348,16217894.0,6646446.0,14552017.0,12954774.0,25.0,25.0,25.0,Sweden,Sweden,Sweden,...,2580,2580,47695.0,832525.0,16010.0,4.0,3.0,3.0,2007,0
